# HappyRobot Load Management API Testing

This notebook provides examples of how to interact with the HappyRobot backend API for load management.

## Features:
- Create new loads
- Update existing loads
- Assign loads (set status to agreed)
- List all loads
- Get specific load details
- Delete loads

## Prerequisites:
- Backend running on port 8000
- Valid API key (if authentication is enabled)


In [1]:
# Import required libraries
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, Any, Optional
import uuid


## Configuration

Set your backend URL and API key here:


In [6]:
# Configuration
BASE_URL = "http://localhost:8000"  # Change this to your backend URL
API_KEY = "HapRob-OTVHhErcXLu2eKkUMP6lDtrd8UNi61KZo4FvGALqem0NoJO1uWlz7OywCN0BNoNaG2x5Y"  # Replace with your actual API key

# Headers for API requests
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "X-API-Key": API_KEY  # API key header
}

print(f"Backend URL: {BASE_URL}")
print(f"API Key: {API_KEY[:10]}..." if len(API_KEY) > 10 else f"API Key: {API_KEY}")


Backend URL: http://localhost:8000
API Key: HapRob-OTV...


## Helper Functions


In [3]:
def make_request(method: str, endpoint: str, data: Optional[Dict] = None) -> Dict[str, Any]:
    """
    Make an API request to the backend
    
    Args:
        method: HTTP method (GET, POST, PATCH, DELETE)
        endpoint: API endpoint (e.g., '/shipments')
        data: Request data for POST/PATCH requests
    
    Returns:
        Response data as dictionary
    """
    url = f"{BASE_URL}{endpoint}"
    
    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=data)
        elif method.upper() == "PATCH":
            response = requests.patch(url, headers=headers, json=data)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=headers)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        
        # Check if request was successful
        if response.status_code in [200, 201, 204]:
            if response.status_code == 204:  # No content
                return {"success": True, "message": "Operation completed successfully"}
            return {"success": True, "data": response.json()}
        else:
            return {
                "success": False, 
                "error": f"HTTP {response.status_code}",
                "message": response.text
            }
            
    except requests.exceptions.RequestException as e:
        return {"success": False, "error": "Request failed", "message": str(e)}

def health_check() -> bool:
    """Check if the backend is healthy"""
    result = make_request("GET", "/health")
    if result["success"]:
        print("✅ Backend is healthy")
        return True
    else:
        print(f"❌ Backend health check failed: {result.get('message', 'Unknown error')}")
        return False

def print_response(result: Dict[str, Any], title: str = "Response"):
    """Pretty print API response"""
    print(f"\n{'='*50}")
    print(f"{title}")
    print(f"{'='*50}")
    
    if result["success"]:
        print("✅ Success!")
        if "data" in result:
            print(json.dumps(result["data"], indent=2, default=str))
        elif "message" in result:
            print(result["message"])
    else:
        print("❌ Failed!")
        print(f"Error: {result.get('error', 'Unknown error')}")
        print(f"Message: {result.get('message', 'No message')}")
    print(f"{'='*50}\n")


## 1. Health Check

First, let's verify that the backend is running and accessible:


In [4]:
# Check backend health
health_check()


✅ Backend is healthy


True

## 2. Create a New Load

Create a new load with all the required and optional fields:


In [8]:
# Create a new load
def create_load(load_data: Dict[str, Any]) -> Dict[str, Any]:
    """Create a new load"""
    return make_request("POST", "/shipments", load_data)

# Example load data
new_load = {
    "load_id": f"LD-NOTEBOOK-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    "origin": "Los Angeles, CA",
    "destination": "New York, NY",
    "pickup_datetime": (datetime.now() + timedelta(days=1)).isoformat() + "Z",
    "delivery_datetime": (datetime.now() + timedelta(days=3)).isoformat() + "Z",
    "equipment_type": "Dry Van",
    "loadboard_rate": 2500.50,
    "weight": 1500.0,
    "commodity_type": "Electronics",
    "num_of_pieces": 100,
    "miles": 2800.0,
    "dimensions": "48x40x60 in",
    "notes": "Created from Jupyter notebook - handle with care"
}

print("Creating new load...")
print(f"Load ID: {new_load['load_id']}")
print(f"Route: {new_load['origin']} → {new_load['destination']}")

result = create_load(new_load)
print_response(result, "Create Load")


Creating new load...
Load ID: LD-NOTEBOOK-20250911-045201
Route: Los Angeles, CA → New York, NY

Create Load
❌ Failed!
Error: HTTP 401
Message: {"detail":"Invalid API key"}



## 3. Assign a Load (Set Status to Agreed)

This is the specific operation you requested - changing a load status to "assigned" (agreed) with the required agreed_price and carrier_description:


In [ ]:
# First, let's get all loads to see what we have
def get_all_loads() -> Dict[str, Any]:
    """Get all loads"""
    return make_request("GET", "/shipments")

# Get all loads
all_loads_result = get_all_loads()
print_response(all_loads_result, "All Loads")

# Find a load to assign (preferably one that's not already assigned)
if all_loads_result["success"] and all_loads_result["data"]:
    loads = all_loads_result["data"]
    
    # Find a pending load to assign
    pending_loads = [load for load in loads if load["status"] == "pending"]
    
    if pending_loads:
        load_to_assign = pending_loads[0]
        print(f"\n🎯 Found load to assign: {load_to_assign['load_id']}")
        print(f"   Route: {load_to_assign['origin']} → {load_to_assign['destination']}")
        print(f"   Current status: {load_to_assign['status']}")
        
        # Assign the load with required fields
        assign_data = {
            "status": "agreed",
            "agreed_price": 2800.00,  # Required when status is agreed
            "carrier_description": "ABC Transport Co."  # Required when status is agreed
        }
        
        print(f"\n📝 Assigning load with:")
        print(f"   Agreed Price: ${assign_data['agreed_price']}")
        print(f"   Carrier: {assign_data['carrier_description']}")
        
        # Make the PATCH request to assign the load
        assign_result = make_request("PATCH", f"/shipments/{load_to_assign['id']}", assign_data)
        print_response(assign_result, "Assign Load")
        
    else:
        print("\n⚠️ No pending loads found to assign")
        print("All loads are already assigned or no loads exist")
        
        # Show current loads status
        if loads:
            print("\n📊 Current loads status:")
            for load in loads:
                print(f"   {load['load_id']}: {load['status']}")
else:
    print("❌ Could not retrieve loads")


## 4. Direct cURL Command for Assigning Loads

Here's the exact cURL command you can use to assign a load:


In [ ]:
# Generate the exact cURL command for assigning a load
def generate_curl_command(load_id: str, agreed_price: float, carrier_description: str):
    """Generate cURL command for assigning a load"""
    
    curl_command = f'''curl -X PATCH "http://localhost:8000/shipments/{load_id}" \\
  -H "Content-Type: application/json" \\
  -H "X-API-Key: {API_KEY}" \\
  -d '{{
    "status": "agreed",
    "agreed_price": {agreed_price},
    "carrier_description": "{carrier_description}"
  }}' '''
    
    return curl_command

# Example: If you have a specific load ID, use this
example_load_id = "your-load-uuid-here"  # Replace with actual load ID
example_agreed_price = 2800.00
example_carrier = "ABC Transport Co."

print("🔧 cURL Command for Assigning a Load:")
print("=" * 60)
print(generate_curl_command(example_load_id, example_agreed_price, example_carrier))
print("=" * 60)

print("\n📝 Instructions:")
print("1. Replace 'your-load-uuid-here' with the actual load ID")
print("2. Adjust the agreed_price and carrier_description as needed")
print("3. Run the command in your terminal")

# If we have a real load from the previous cell, show the actual command
if 'load_to_assign' in locals():
    print(f"\n🎯 Actual cURL command for load {load_to_assign['load_id']}:")
    print("=" * 60)
    actual_curl = generate_curl_command(
        load_to_assign['id'], 
        2800.00, 
        "ABC Transport Co."
    )
    print(actual_curl)
    print("=" * 60)
